# OpenAI embeddings model which is mutli-language
### Czech test

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
import os
import openai

# Set OpenAI API credentials
openai.api_type = "azure"
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("BASE_URL")
openai.api_version = "2023-07-01-preview"
deployment_name = os.getenv("DEPLOYMENT_NAME")

In [3]:
# Import list of texts
import mytexts

# Create a pandas DataFrame from the list of words
import pandas as pd

df = pd.DataFrame(mytexts.texts_cz, columns=["text"])

def get_embedding(text):
   return openai.Embedding.create(input = [text], deployment_id=deployment_name)['data'][0]['embedding']

df['embedding'] = df['text'].apply(lambda x: get_embedding(x))

In [4]:
from openai.embeddings_utils import cosine_similarity
import numpy as np

def get_similar(source_texts, source_embeddings, target_embedding):
    df = pd.DataFrame(source_texts, columns=['text'])
    df['embedding'] = source_embeddings
    df['similarities'] = df['embedding'].apply(lambda x: cosine_similarity(x, target_embedding))
    return df.sort_values(by='similarities', ascending=False).drop(columns=['embedding'])

get_similar(df['text'], df['embedding'], get_embedding('Karel IV.'))

,text,similarities
0,"Král: Mužský panovník, který typicky získává s...",0.826982
1,"Královna: Ženský panovník, který typicky získá...",0.820007
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.791113
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.787196
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.786764
4,"Pes: domestikovaná šelma, která provází člověk...",0.763578
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.759550
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.752823
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.751189


In [5]:
get_similar(df['text'], df['embedding'], get_embedding('woman sitting on throne'))

,text,similarities
1,"Královna: Ženský panovník, který typicky získá...",0.820404
0,"Král: Mužský panovník, který typicky získává s...",0.806356
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.780668
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.762109
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.757530
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.744676
4,"Pes: domestikovaná šelma, která provází člověk...",0.737916
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.726672
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.709054


In [12]:
get_similar(df['text'], df['embedding'], get_embedding('Bowling je halový sport, ve kterém se hráč snaží hozenou koulí srazit co možná nejvyšší počet kuželek na konci dráhy.'))

,text,similarities
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.859111
4,"Pes: domestikovaná šelma, která provází člověk...",0.812804
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.808812
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.789860
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.777007
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.776292
0,"Král: Mužský panovník, který typicky získává s...",0.775597
1,"Královna: Ženský panovník, který typicky získá...",0.774009
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.768294


In [10]:
get_similar(df['text'], df['embedding'], get_embedding('人类经常饲养的四足兽'))

,text,similarities
4,"Pes: domestikovaná šelma, která provází člověk...",0.806217
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.786069
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.759606
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.730455
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.724972
0,"Král: Mužský panovník, který typicky získává s...",0.724391
1,"Královna: Ženský panovník, který typicky získá...",0.722194
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.717882
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.711771


In [8]:
# Vector calculations
positive_text = ["Král je dědičný titul panovníka monarchie. V minulosti byl král nejvyšší autoritou v zemi a měl mnoho privilegi",
                 "Žena je podstatné jméno ženského rodu, které označuje dospělou samici člověka. V obecném jazyce se slovo žena používá také jako synonymum pro paní nebo dáma"]
negative_text = ["Muž je podstatné jméno mužského rodu, které označuje dospělého samce člověka. V obecném jazyce se slovo muž používá také jako synonymum pro chlap nebo kluk."]
vector = np.zeros(1536)
vector = sum([np.array(get_embedding(text)) for text in positive_text])
vector = vector + sum([-np.array(get_embedding(text)) for text in negative_text])

get_similar(df['text'], df['embedding'], vector)

,text,similarities
1,"Královna: Ženský panovník, který typicky získá...",0.839392
0,"Král: Mužský panovník, který typicky získává s...",0.806693
3,"Žena: Samice člověka, ženská lidská bytost, kt...",0.756048
7,"Hrad: opevněné feudální sídlo1, které se stavě...",0.741525
8,"Kuželky: hovorové označení pro sportovní hru, ...",0.729424
5,Lev: savec čeledi kočkovitých a jeden z pěti d...,0.720193
4,"Pes: domestikovaná šelma, která provází člověk...",0.711233
6,Guláš: pokrm z kousků masa dušeného na cibulce...,0.708566
2,"Muž: Samec člověka, mužská lidská bytost, kter...",0.688603
